In [1]:
%load_ext autoreload
%autoreload 2

import json

from dac_costing.model import DacModel, DacSection, BatterySection, EnergySection

## Model Parameters

Here we open the default model parameters. We can modify these as needed...

In [2]:
with open('../dac_costing/data/parameters.json', 'r') as f:
    params = json.load(f)

In [3]:
from uncertainties import ufloat
stdev = 0.1

def cast_to_ufloat(d):
    u = {}
    for p, val in d.items():
        if isinstance(val, dict):
            u[p] = cast_to_ufloat(val)
        if isinstance(val, float):
            u[p] = ufloat(val, val*stdev, tag=p)
        else:
            u[p] = val
    return u

params = cast_to_ufloat(params)

## C1 - Natural Gas

In [4]:
from dac_costing.model import NgThermalSection

In [5]:
params['Base Energy Requirement [MW]'] = 46.6  # ='Report Data'!C58
params['Required Thermal Energy [GJ/tCO2]'] = 6.64
params['Total Capex [$]'] = 1029  # =+'Report Data'!C21

electric = EnergySection(source='NGCC w/ CCS', battery=None, params=params)
electric.compute().series

thermal = NgThermalSection(source='Advanced NGCC', battery=None, params=params)

dac = DacSection(params=params)
dac_all = DacModel(electric=electric, thermal=thermal, dac=dac, params=params)
dac_all.compute().series

Total Capital Cost [M$]          1479+/-24
Capital Recovery [$/tCO2eq]       156+/-12
Fixed O&M [$/tCO2eq]               36.3977
Variable O&M [$/tCO2eq]        6.14+/-0.21
Natural Gas Cost [$/tCO2]       30.5+/-3.2
Total Cost [$/tCO2]               229+/-13
dtype: object

## C2 - Electric Kiln (Solar)

In [6]:
params['Base Energy Requirement [MW]'] = 38  # ='Report Data'!C64
params['Total Capex [$]'] = 936.01  # ='Report Data'!H27

ebattery = BatterySection(params)
electric = EnergySection(source='Solar', battery=ebattery, params=params)

params['Base Energy Requirement [MW]'] = 234  # =F18
tbattery = BatterySection(params=params)
thermal = EnergySection(source='Solar', battery=tbattery, params=params)

dac = DacSection(params=params)
dac_all = DacModel(electric=electric, thermal=thermal, dac=dac, params=params)
dac_all.compute().series

Total Capital Cost [M$]        3759+/-32
Capital Recovery [$/tCO2eq]     397+/-28
Fixed O&M [$/tCO2eq]             72.2679
Variable O&M [$/tCO2eq]          4.0+/-0
Natural Gas Cost [$/tCO2]        0.0+/-0
Total Cost [$/tCO2]             473+/-28
dtype: object

## C3 - Electric Kiln (Nuclear)

In [7]:
params['Base Energy Requierement [MW]'] = 38  # ='Report Data'!C64
params['Base Energy Requierement [MW]'] = 234  # =F18
params['Total Capex [$]'] = 936.01  # ='Report Data'!H27

electric = EnergySection(source='Advanced Nuclear', battery=None, params=params)
thermal = EnergySection(source='Advanced Nuclear', battery=None, params=params)
dac = DacSection(params=params)

dac_all = DacModel(electric=electric, thermal=thermal, dac=dac, params=params)
dac_all.compute().series

Total Capital Cost [M$]        (8.42+/-0.23)e+03
Capital Recovery [$/tCO2eq]      (8.9+/-0.8)e+02
Fixed O&M [$/tCO2eq]                      138.44
Variable O&M [$/tCO2eq]               12.7+/-0.9
Natural Gas Cost [$/tCO2]               132+/-19
Total Cost [$/tCO2]            (1.17+/-0.08)e+03
dtype: object

## C4 - Electric Kiln (Wind)

In [8]:
params['Base Energy Requierement [MW]'] = 38  # ='Report Data'!C64
params['Total Capex [$]'] = 936.01  # ='Report Data'!H27

ebattery = BatterySection(params=params)
electric = EnergySection(source='Wind', battery=ebattery, params=params)

params['Base Energy Requierement [MW]'] = 234  # =F18
tbattery = BatterySection(params=params)
thermal = EnergySection(source='Wind', battery=tbattery, params=params)

dac = DacSection(params=params)

dac_all = DacModel(electric=electric, thermal=thermal, dac=dac, params=params)
dac_all.compute().series

Total Capital Cost [M$]        (4.35+/-0.04)e+03
Capital Recovery [$/tCO2eq]             460+/-33
Fixed O&M [$/tCO2eq]                     90.7114
Variable O&M [$/tCO2eq]                  4.0+/-0
Natural Gas Cost [$/tCO2]                0.0+/-0
Total Cost [$/tCO2]                     555+/-33
dtype: object